# Trabalho Final de Mineração de Dados

## Importando bibliotecas

In [1]:
import string

import nltk
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
from imblearn.under_sampling import RandomUnderSampler
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\paulo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## 1 - Fase de Preparação dos Dados
***

Carregando o arquivo original de classes.

In [2]:
classes_original = pd.read_excel("dados/SistemaDeClassificacao.xlsx", "Select viw_classificacao_arvore")
classes_original.head()

,,COD_CLASSE,DES_NOME_PREFERIDO,COD_CLASSE_PAI,NUM_NIVEL,CYCLE,TREE,PATH,QTD_FILHOS,DES_NIVEL1,DES_NIVEL2,DES_NIVEL3,DES_NIVEL4
0,1,33254494,Classificação Temática Unificada,NaN,1,0,Classificação Temática Unificada,Classificação Temática Unificada,202,NaN,NaN,NaN,NaN
1,2,33809814,Temas Exclusivos de Pronunciamentos,33254494.0,2,0,-- Temas Exclusivos de Pronunciamentos,Classificação Temática Unificada / Temas Exclu...,22,Temas Exclusivos de Pronunciamentos,NaN,NaN,NaN
2,3,33809634,Meio Ambiente,33254494.0,2,0,-- Meio Ambiente,Classificação Temática Unificada / Meio Ambiente,11,Meio Ambiente,NaN,NaN,NaN
3,4,33809514,"Soberania, Defesa Nacional e Ordem Pública",33254494.0,2,0,"-- Soberania, Defesa Nacional e Ordem Pública","Classificação Temática Unificada / Soberania, ...",7,"Soberania, Defesa Nacional e Ordem Pública",NaN,NaN,NaN
4,5,33808912,Política Social,33254494.0,2,0,-- Política Social,Classificação Temática Unificada / Política So...,37,Política Social,NaN,NaN,NaN


Filtra o arquivo de classes original para exibir apenas aquelas que serão raiz na classificação da ementa. Essa classes possuem nível 2 na tabela original.

In [3]:
classes_raiz = classes_original.query("NUM_NIVEL == 2").filter(["COD_CLASSE", "DES_NOME_PREFERIDO"])
classes_raiz = classes_raiz.sort_values("DES_NOME_PREFERIDO")
classes_raiz.reset_index(drop=True, inplace=True)
classes_raiz.rename(columns={"DES_NOME_PREFERIDO": "DES_CLASSE", "index": "INDICE"}, inplace=True)
classes_raiz

,COD_CLASSE,DES_CLASSE
0,33685789,Administração Pública
1,33769167,Economia e Desenvolvimento
2,33805317,Honorífico
3,33805137,Infraestrutura
4,33805362,Jurídico
5,33809634,Meio Ambiente
6,33768972,Organização do Estado
7,33260515,Orçamento Público
8,33808912,Política Social
9,33809514,"Soberania, Defesa Nacional e Ordem Pública"


Carrega o dataset contendo as normas que já foram classificadas. Com os dados carregados, cria uma coluna derivada (DES_CLASSE_RAIZ) a partir da árvore de classes (DES_CLASSE_HIERARQUIA), da qual a informação de classe raiz é extraída.

In [4]:
normas_classificadas_original = pd.read_excel(
    "dados/ClassificacaoDeLeisOrdinarias-LeisComplementares-e-DecretosNumerados-Desde1900.xlsx",
    "Select mvw_u03_prc_doc_tema")
normas_classificadas_original["DES_CLASSE_RAIZ"] = normas_classificadas_original["DES_CLASSE_HIERARQUIA"].apply(
    lambda hierarquia: hierarquia.split(" / ")[1])
normas_classificadas_original.head()

,,COD_PRC_DOC_TEMA,COD_PROCESSO_DOCUMENTO,COD_CLASSE,DES_CLASSE,DES_CLASSE_HIERARQUIA,DES_CLASSE_RAIZ
0,1,36155183,386343,33805827,Crédito Suplementar,Classificação Temática Unificada / Orçamento P...,Orçamento Público
1,2,36192020,386579,33805287,Rádio e TV,Classificação Temática Unificada / Infraestrut...,Infraestrutura
2,3,36155185,387419,33805827,Crédito Suplementar,Classificação Temática Unificada / Orçamento P...,Orçamento Público
3,4,36192056,387832,33805287,Rádio e TV,Classificação Temática Unificada / Infraestrut...,Infraestrutura
4,5,36155187,388197,33805827,Crédito Suplementar,Classificação Temática Unificada / Orçamento P...,Orçamento Público


Carrega os dados de todas as normas (classificadas e não classificadas) e aplica as transformações iniciais.

In [5]:
normas_original = pd.read_excel(
    "dados/LeisOrdinarias-LeisComplementare-e-DecretosNumeradosComClassificacaoDesde1900.xlsx",
    "Select mvw_s01_documento")
normas_original.rename(columns={"DBMS_LOB.SUBSTR(S01.TXT_EMENTA": "TXT_EMENTA"}, inplace=True)
normas_original.drop(columns="   ", inplace=True)
normas_original["ANO"] = normas_original["DES_NOME_PREFERIDO"].apply(lambda x : x[len(x) - 4:])
normas_original.head()

,COD_DOCUMENTO,DES_NOME_PREFERIDO,DES_NOMES_ALTERNATIVOS,TXT_EMENTA,ANO
0,35345364,Lei nº 14.263 de 22/12/2021,LEI-14263-2021-12-22,Abre ao Orçamento da Seguridade Social da Uniã...,2021
1,26247104,Lei nº 13.486 de 03/10/2017,LEI-13486-2017-10-03,"Altera o art. 8º da Lei nº 8.078, de 11 de set...",2017
2,27445746,Lei nº 13.701 de 06/08/2018,LEI-13701-2018-08-06,Cria o cargo de natureza especial de Intervent...,2018
3,36348502,Lei nº 14.447 de 09/09/2022,LEI-14447-2022-09-09,Altera os limites da Floresta Nacional de Bras...,2022
4,32103727,Lei nº 13.988 de 14/04/2020,LEI-13988-2020-04-14,Dispõe sobre a transação nas hipóteses que esp...,2020


In [6]:
normas_original.shape[0]

26959

Com os datasets necessários já carregados, cria-se um novo dataset a partir do dataset contendo todas as leis, incluindo-se a informação das classes contidadas nas normas do dataset de normas classificadas.

In [7]:
normas = normas_original.merge(normas_classificadas_original.filter(["COD_PROCESSO_DOCUMENTO", "DES_CLASSE_RAIZ"]),
                               left_on="COD_DOCUMENTO", right_on="COD_PROCESSO_DOCUMENTO", how="left")
normas = normas.merge(classes_raiz, left_on="DES_CLASSE_RAIZ", right_on="DES_CLASSE", how="left")
normas.drop(columns=["COD_PROCESSO_DOCUMENTO", "DES_CLASSE_RAIZ"], inplace=True)
normas.head()

,COD_DOCUMENTO,DES_NOME_PREFERIDO,DES_NOMES_ALTERNATIVOS,TXT_EMENTA,ANO,COD_CLASSE,DES_CLASSE
0,35345364,Lei nº 14.263 de 22/12/2021,LEI-14263-2021-12-22,Abre ao Orçamento da Seguridade Social da Uniã...,2021,33260515.0,Orçamento Público
1,26247104,Lei nº 13.486 de 03/10/2017,LEI-13486-2017-10-03,"Altera o art. 8º da Lei nº 8.078, de 11 de set...",2017,33808912.0,Política Social
2,27445746,Lei nº 13.701 de 06/08/2018,LEI-13701-2018-08-06,Cria o cargo de natureza especial de Intervent...,2018,33768972.0,Organização do Estado
3,36348502,Lei nº 14.447 de 09/09/2022,LEI-14447-2022-09-09,Altera os limites da Floresta Nacional de Bras...,2022,33685789.0,Administração Pública
4,36348502,Lei nº 14.447 de 09/09/2022,LEI-14447-2022-09-09,Altera os limites da Floresta Nacional de Bras...,2022,33809634.0,Meio Ambiente


In [8]:
normas.shape[0]

27743

Limpa os registros duplicados de normas que possuem mais de uma classe folha, mas que tenham mesma classe raiz e exibe as leis que possuem mais de uma classe folha, mas que tenham classes raiz diferentes.

In [9]:
normas.drop_duplicates(inplace=True)
normas.reset_index(drop=True, inplace=True)
temp = normas[["COD_DOCUMENTO", "COD_CLASSE", "DES_CLASSE"]].groupby("COD_DOCUMENTO")
temp.filter(lambda x: len(x) > 1)

,COD_DOCUMENTO,COD_CLASSE,DES_CLASSE
3,36348502,33685789.0,Administração Pública
4,36348502,33809634.0,Meio Ambiente
5,36348502,33769167.0,Economia e Desenvolvimento
18,36062349,33808912.0,Política Social
19,36062349,33769167.0,Economia e Desenvolvimento
...,...,...,...
26904,36032872,33808912.0,Política Social
26905,35556312,33685789.0,Administração Pública
26906,35556312,33808912.0,Política Social
26910,35396946,33809514.0,"Soberania, Defesa Nacional e Ordem Pública"


Tamanho da base tratada de normas.

In [10]:
normas.shape[0]

27294

Total de classificações feitas em normas.

In [11]:
normas_classificadas = normas.query("not COD_CLASSE.isnull()")
normas_classificadas.reset_index(drop=True, inplace=True)
normas_classificadas.shape[0]

17438

Número distinto de normas classificadas.

In [12]:
len(normas_classificadas["COD_DOCUMENTO"].unique())

17103

Quantidade de normas a classificar.

In [13]:
def criar_dataset_normas_nao_classificadas():
    normas_nao_classificadas = normas.query("COD_CLASSE.isnull()")
    normas_nao_classificadas.reset_index(drop=True, inplace=True)
    normas_nao_classificadas = normas_nao_classificadas.drop(columns=["COD_CLASSE", "DES_CLASSE"])

    return normas_nao_classificadas

normas_nao_classificadas = criar_dataset_normas_nao_classificadas()
normas_nao_classificadas.shape[0]

9856

Verificação da distribuição das classes (em %).

In [14]:
frequencia = normas_classificadas["DES_CLASSE"].value_counts()
percentual = normas_classificadas["DES_CLASSE"].value_counts(normalize=True) * 100
distribuicao_classes = pd.DataFrame({'Frequência': frequencia, 'Percentual (%)': percentual})
distribuicao_classes.sort_values(by="Frequência", ascending=False)

,Frequência,Percentual (%)
Orçamento Público,11192,64.181672
Infraestrutura,2508,14.382383
Política Social,877,5.029246
Administração Pública,783,4.490194
Honorífico,671,3.847918
Economia e Desenvolvimento,644,3.693084
Jurídico,381,2.184884
"Soberania, Defesa Nacional e Ordem Pública",166,0.951944
Organização do Estado,130,0.745498
Meio Ambiente,86,0.493176


## 2 - Estruturação do texto
***

Definição das funções para a tarefa de criação das matrizes de documentos (texto da ementa)

In [15]:
# Divide um texto em tokens utilizando a biblioteca de NLP nltk
def tokenizador(texto):
    texto = str(texto).lower()
    lista_alfanumerica = []

    for token_valido in nltk.word_tokenize(texto, language="portuguese"):
        # Se for caracter de pontuação ou for stopword
        if token_valido in string.punctuation:
            continue

        lista_alfanumerica.append(token_valido)

    return lista_alfanumerica

In [16]:
# Combinação de vetores por soma
def combinacao_vetores(palavras_numeros, skipgram):
    vetor_resultante = np.zeros(300)

    for pn in palavras_numeros:
        try:
            vetor_resultante += skipgram.get_vector(pn)
        except KeyError:
            if pn.isnumeric():
                pn = "0" * len(pn)  # "0, 00, 000, 0000, etc. dependendo do tamanho de caracteres do número
                vetor_resultante += skipgram.get_vector(pn)
            else:
                vetor_resultante += skipgram.get_vector("unknown")

    return vetor_resultante

In [17]:
def matriz_word_embeddings(textos, skipgram):
    x = len(textos)
    y = 300  # número de dimensões do skipgram (skip300)
    matriz = np.zeros((x, y))

    for i in range(x):
        palavras_numeros = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_vetores(palavras_numeros, skipgram)

    return matriz

Carrega o word embeddings (Word2Vec - Skipgram) pré-treinado pelo NILC (Núcleo Interinstitucional de Linguística Computacional).

In [18]:
# Carrega o word embeddings skip-gram
skipgram = KeyedVectors.load_word2vec_format("dados/skip_s300.txt")

Criação da matriz de documentos do corpus (conjunto de ementas), aqui representada por matrizes de word embeddings (cada ementa é vetorizada assim como o word embeddings pré-treinado). Também é feita a separação das classes (labels).

In [19]:
X = matriz_word_embeddings(normas_classificadas["TXT_EMENTA"], skipgram)
y = normas_classificadas["DES_CLASSE"]

X_nao_classificadas = matriz_word_embeddings(normas_nao_classificadas["TXT_EMENTA"], skipgram)

## 3 - Treinamento e predição do modelo de Machine Learning para classificação

### 3.1 - Comparação de modelos - Método _hold-out_

Separação inicial dos dados, utilizando-se o método _hold-out_ para a avaliação inicial de algoritmos de ML. A proporção utilizada é de 70% para treino e 30% para testes.

In [20]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

Modelos de classificação a serem comparados.

In [21]:
modelos = {"Dummy": DummyClassifier(random_state=42),
           "BernoulliNB": BernoulliNB(binarize=np.median(X_treino)),
           "KNN 1": KNeighborsClassifier(n_neighbors=1),
           "KNN 3": KNeighborsClassifier(n_neighbors=3),
           "KNN 5": KNeighborsClassifier(),
           "KNN 11": KNeighborsClassifier(n_neighbors=11),
           "DecisionTree": DecisionTreeClassifier(class_weight="balanced", random_state=42),
           "SVC": SVC(class_weight="balanced", random_state=42),
           "RandomForest": RandomForestClassifier(class_weight="balanced", random_state=42),
           "LogisticRegression": LogisticRegression(max_iter=6000, class_weight="balanced", multi_class="ovr", random_state=42)}

Definindo a função para execução dos modelos, utilizando-se o método _hold-out_.

In [22]:
def executar_modelo_holdout(modelo, X_treino, y_treino, X_teste, y_teste):
    modelo.fit(X_treino, y_treino)
    y_previsto = modelo.predict(X_teste)
    return accuracy_score(y_teste, y_previsto)

Executando os modelos de Machine Learning para avaliação.

In [23]:
for nome, modelo in modelos.items():
    score = executar_modelo_holdout(modelo, X_treino, y_treino, X_teste, y_teste)
    print(f"{nome}: {score * 100:0.2f}")

Dummy: 64.18
BernoulliNB: 81.19
KNN 1: 88.15
KNN 3: 89.05
KNN 5: 89.18
KNN 11: 89.01
DecisionTree: 82.32
SVC: 89.07
RandomForest: 88.11
LogisticRegression: 90.81


Treinamento e predição do melhor classificador encontrado (LogisticRegression) com o método _hold-out_.

In [24]:
lr = LogisticRegression(max_iter=6000, class_weight="balanced", random_state=42, multi_class="ovr")
lr.fit(X_treino, y_treino)
y_previsto = lr.predict(X_teste)

#### 3.1.1 - Avaliando o modelo de maior acurácia

Exibe os relatórios de desempenho do modelo de maior acurácia (LogisticRegression).

In [25]:
acuracia = accuracy_score(y_teste, y_previsto)
print("Acurácia LogisticRegression:", round(acuracia, 2))

cr = classification_report(y_teste, y_previsto)
print(cr)

Acurácia LogisticRegression: 0.91
                                            precision    recall  f1-score   support

                     Administração Pública       0.63      0.63      0.63       235
                Economia e Desenvolvimento       0.54      0.61      0.57       193
                                Honorífico       0.95      0.88      0.91       201
                            Infraestrutura       0.97      0.95      0.96       753
                                  Jurídico       0.54      0.52      0.53       114
                             Meio Ambiente       0.43      0.35      0.38        26
                     Organização do Estado       0.26      0.33      0.29        39
                         Orçamento Público       1.00      0.99      0.99      3358
                           Política Social       0.60      0.64      0.62       263
Soberania, Defesa Nacional e Ordem Pública       0.32      0.30      0.31        50

                                  accura

#### 3.1.2 - Criando _baseline_ do dataset com redução das classes majoritárias

Para tentar diminuir o desbalanceamento dos dados, o número de amostras das duas classes majoritárias (Orçamento Público e Infraestrutura) será diminuído para 1000 cada.

In [26]:
rus = RandomUnderSampler(sampling_strategy={"Orçamento Público": 1000, "Infraestrutura": 1000}, random_state=42)
X_res, y_res = rus.fit_resample(X, y)

frequencia_res = y_res.value_counts()
percentual_res = y_res.value_counts(normalize=True) * 100
distribuicao_classes_res = pd.DataFrame({'Frequência': frequencia_res, 'Percentual (%)': percentual_res})
distribuicao_classes_res.sort_values(by="Frequência", ascending=False)

,Frequência,Percentual (%)
Infraestrutura,1000,17.427675
Orçamento Público,1000,17.427675
Política Social,877,15.284071
Administração Pública,783,13.645870
Honorífico,671,11.693970
Economia e Desenvolvimento,644,11.223423
Jurídico,381,6.639944
"Soberania, Defesa Nacional e Ordem Pública",166,2.892994
Organização do Estado,130,2.265598
Meio Ambiente,86,1.498780


Dividindo o dataset na mesma proporção treino/teste anterior (70%/30%).

In [27]:
X_treino_res, X_teste_res, y_treino_res, y_teste_res = train_test_split(X_res, y_res, test_size=0.3, stratify=y_res, random_state=42)

Treinamento, predição e avaliação do novo modelo (LogisticRegression) com dataset reduzido.

In [28]:
lr_res = LogisticRegression(max_iter=6000, class_weight="balanced", random_state=42)
lr_res.fit(X_treino_res, y_treino_res)
y_previsto_res = lr_res.predict(X_teste_res)

In [29]:
acuracia_res = accuracy_score(y_teste_res, y_previsto_res)
print("Acurácia LogisticRegression:", round(acuracia_res, 2))

cr_res = classification_report(y_teste_res, y_previsto_res)
print(cr_res)

Acurácia LogisticRegression: 0.71
                                            precision    recall  f1-score   support

                     Administração Pública       0.59      0.53      0.56       235
                Economia e Desenvolvimento       0.57      0.53      0.55       193
                                Honorífico       0.89      0.93      0.91       202
                            Infraestrutura       0.94      0.93      0.93       300
                                  Jurídico       0.48      0.56      0.52       114
                             Meio Ambiente       0.36      0.35      0.35        26
                     Organização do Estado       0.24      0.49      0.32        39
                         Orçamento Público       0.98      0.97      0.98       300
                           Política Social       0.57      0.52      0.55       263
Soberania, Defesa Nacional e Ordem Pública       0.28      0.30      0.29        50

                                  accura

### 3.2 - Comparação de modelos - _10 Fold Cross-Validation_

Definindo a função para comparação dos modelos, utilizando-se o método _10 Fold cross-validation_.

In [30]:
def executar_modelo_cross_validation(modelo, X, y, splits=10):
    cv = StratifiedKFold(n_splits=splits, shuffle=True, random_state=42)  # shuffle gera uma forma aleatória dos dados
    scores = cross_val_score(modelo, X, y, cv=cv)
    return scores, modelo

In [31]:
def imprimir_scores(nome, scores):
    media = scores.mean()
    desvio_padrao = scores.std()
    print(f"{nome}: {media * 100:0.2f} [{(media - 2 * desvio_padrao) * 100:0.2f}, {(media + 2 * desvio_padrao) * 100:0.2f}]")

Executando os modelos de Machine Learning para avaliação.

In [32]:
for nome, modelo in modelos.items():
    scores, _ = executar_modelo_cross_validation(modelo, X, y)
    imprimir_scores(nome, scores)

Dummy: 64.18 [64.13, 64.23]
BernoulliNB: 80.45 [78.71, 82.19]
KNN 1: 88.75 [87.60, 89.91]
KNN 3: 89.40 [88.37, 90.43]
KNN 5: 89.70 [88.49, 90.91]
KNN 11: 89.84 [88.77, 90.92]
DecisionTree: 82.81 [81.39, 84.24]
SVC: 89.59 [88.29, 90.88]
RandomForest: 88.65 [88.11, 89.18]
LogisticRegression: 90.84 [89.93, 91.75]


Treinamento, predição e avaliação com o melhor classificador encontrado (LogisticRegression) utilizando-se o método de _cross-validation_.

In [33]:
lr = LogisticRegression(max_iter=6000, class_weight="balanced", random_state=42, multi_class="ovr")

scores, lr = executar_modelo_cross_validation(lr, X, y)
imprimir_scores("LogisticRegression", scores)

lr.fit(X, y)
y_previsto = lr.predict(X)

LogisticRegression: 90.84 [89.93, 91.75]


In [34]:
cr = classification_report(y, y_previsto)
print(cr)

                                            precision    recall  f1-score   support

                     Administração Pública       0.76      0.73      0.74       783
                Economia e Desenvolvimento       0.62      0.74      0.67       644
                                Honorífico       0.97      0.98      0.97       671
                            Infraestrutura       0.99      0.96      0.97      2508
                                  Jurídico       0.71      0.77      0.74       381
                             Meio Ambiente       0.77      0.91      0.83        86
                     Organização do Estado       0.56      0.62      0.59       130
                         Orçamento Público       1.00      0.99      1.00     11192
                           Política Social       0.76      0.72      0.74       877
Soberania, Defesa Nacional e Ordem Pública       0.64      0.74      0.69       166

                                  accuracy                           0.94 

## 4 - Predição do conjunto de dados não classificados

Considerando-se que o modelo baseado em Regressão Logística foi o melhor ajustado para o problema em todos os cenários testados, a predição do conjunto de dados não classificados será feita beseado nele. Além disso, para todas as estimativas de classes realizadas, foram incluídas as probabilidades utilizadas pelo algoritmo para realização da cada uma das previsões, permitindo que o especialista possa analisar e decidir se aceita ou não a previsão feita pelo modelo de _Machine Learning_.

In [35]:
# A partir do modelo gerado anteriormente, prevê as classes das normas não classificadas.
y_previsto = lr.predict(X_nao_classificadas)
y_proba = lr.predict_proba(X_nao_classificadas)

# Reinicia o dataset de normas não classificadas para o caso de alguma modificação tenha sido realizada
normas_nao_classificadas = criar_dataset_normas_nao_classificadas()

# Constrói o dataset final com as previsões e probabilidades.
previsoes = pd.DataFrame(y_previsto, columns=["CLASSE_PROVAVEL"])
probabilidades = pd.DataFrame(y_proba)
normas_nao_classificadas = pd.concat([normas_nao_classificadas, probabilidades, previsoes], axis=1)
normas_nao_classificadas = normas_nao_classificadas.melt(id_vars=["COD_DOCUMENTO","DES_NOME_PREFERIDO","DES_NOMES_ALTERNATIVOS","TXT_EMENTA","ANO","CLASSE_PROVAVEL"], var_name="CLASSE_TEMP", value_name="PROBABILIDADE")
normas_nao_classificadas["CLASSE_TEMP"] = normas_nao_classificadas["CLASSE_TEMP"].map(classes_raiz["DES_CLASSE"].to_dict())
normas_nao_classificadas = normas_nao_classificadas.query("CLASSE_PROVAVEL == CLASSE_TEMP")
normas_nao_classificadas.drop(columns="CLASSE_TEMP", inplace=True)
normas_nao_classificadas.sort_values(by="PROBABILIDADE", ascending=False, inplace=True)
normas_nao_classificadas.reset_index(drop=True, inplace=True)
normas_nao_classificadas

,COD_DOCUMENTO,DES_NOME_PREFERIDO,DES_NOMES_ALTERNATIVOS,TXT_EMENTA,ANO,CLASSE_PROVAVEL,PROBABILIDADE
0,545971,Lei nº 4.129 de 27/08/1962,LEI-4129-1962-08-27,"Autoriza o Poder Executivo a abrir, pelo Minis...",1962,Política Social,0.999995
1,540989,Lei nº 72 de 18/07/1935,LEI-72-1935-07-18,"Autoriza a abrir, pelo Ministerio da Viação, o...",1935,Orçamento Público,0.999990
2,551655,Lei nº 9.779 de 19/01/1999,LEI-9779-1999-01-19,"Altera a legislação do Imposto sobre a Renda, ...",1999,Economia e Desenvolvimento,0.999983
3,549729,Lei nº 7.853 de 24/10/1989,"LEI-7853-1989-10-24 , Lei da Pessoa com Defici...",Dispõe sobre o apoio às pessoas portadoras de ...,1989,Política Social,0.999979
4,546721,Lei nº 4.867 de 30/11/1965,LEI-4867-1965-11-30,Concede a pensão especial de Cr$ 66.000 (sesse...,1965,Política Social,0.999976
...,...,...,...,...,...,...,...
9851,548182,Lei nº 6.306 de 15/12/1975,LEI-6306-1975-12-15,Altera o § 2º do art. 26 do Decreto-lei nº 3.3...,1975,Administração Pública,0.175303
9852,542152,Lei nº 617 de 10/02/1949,LEI-617-1949-02-10,Modifica os artigos 4º e 5º do Decreto-Lei n. ...,1949,"Soberania, Defesa Nacional e Ordem Pública",0.173023
9853,541919,Lei nº 502 de 11/09/1937,LEI-502-1937-09-11,Revoga o parágrafo único do art. 33 do decreto...,1937,"Soberania, Defesa Nacional e Ordem Pública",0.170754
9854,547526,Lei nº 5.653 de 27/04/1971,LEI-5653-1971-04-27,"Altera o art. 19 do Decreto-Lei nº 3.200, de 1...",1971,Organização do Estado,0.158356


Exportando para o formato Excel.

In [36]:
normas_nao_classificadas.to_excel("dados/classificacoes_sugeridas.xlsx", "Classificações", index=False, freeze_panes=(1, 0))